In [1]:
from dotenv import load_dotenv
import os
from langchain_community.graphs import Neo4jGraph
from openai import AzureOpenAI

Conectando ao Neo4j e aos Azure OpenAI

In [83]:
load_dotenv()

# Neo4j variables
NEO4J_URL = os.getenv("NEO4J_URL")
NEO4J_USERNAME =os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

#Connecting to the graph
graph = Neo4jGraph(
    url=NEO4J_URL,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

In [84]:
# Conectando o modelo de Embedding

client_small = AzureOpenAI(
  api_key = os.getenv("OPENAI_API_KEY"),
  azure_endpoint = os.getenv("EMBEDDING_SMALL_ENDPOINT"),
  api_version = os.getenv("API_VERSION"))

#client_large = AzureOpenAI(
#  api_key = os.getenv("EMBEDDING_LARGE_API_KEY"),
#  azure_endpoint = os.getenv("EMBEDDING_LARGE_ENDPOINT"),
#  api_version = os.getenv("API_VERSION"))

# The function receives a sencence and returns the embedding (1D numpy array)
def get_embeddings_openAI(text):
    model = "text-embedding-3-small"
    embedding_small = client_small.embeddings.create(input = text, model=model)
    #embedding_large = client_large.embeddings.create(input = [text], model=model)
    return embedding_small.data 

### GRAPH RAG  
  
Definir uma funcao que: 1) recebe um texto; 2)usa o modelo de embedding para transformar o texto em um vetor: 3) Usa o vetor para buscar outros textos do grafo (usando o índice de vetores)   

In [109]:
def busca_grafo_vetor(query_text):
    query_embedding = get_embeddings_openAI(query_text)[0].embedding

    # Buscando no índice de vetores 
    results = graph.query("""
        CALL db.index.vector.queryNodes('Thesis_Embeddings', 15, """ + str(query_embedding) + """)
        YIELD node, score
        MATCH (node)-[:author]-(author)
        RETURN node.uri, node.title, node.abstract, node.created, author.label, score
        """)
    
    return (results)

In [110]:
def busca_grafo_fulltext(query_text):

    results = graph.query("""
        CALL db.index.fulltext.queryNodes("Thesis_fulltext", '""" + query_text + """') 
        YIELD node, score
        MATCH (node)-[:author]-(author)
        RETURN node.uri, node.title, node.abstract, node.created , author.label, score 
        LIMIT 15 
        """)
    return (results)

In [111]:
# Reciprocal Rank Fusion (RRF)
def RFF(rank1, rank2, w_rank1=1.0, w_rank2=1.0):
    k = 60
    score = {}
    title_text = {}

    for p in range(len(rank1)):
        uri = rank1[p]['node.uri']
        score[uri] = 1/(p+1+k)
        title_text[uri] = {'title': rank1[p]['node.title'], 
                           'text': rank1[p]['node.abstract'], 
                           'author': rank1[p]['author.label'],
                           'created': rank1[p]['node.created']}

    for p in range(len(rank2)):
        uri = rank2[p]['node.uri']
        if uri not in score:
            score[uri] = 1/(p+1+k)
            title_text[uri] = {'title': rank2[p]['node.title'], 
                               'text': rank2[p]['node.abstract'], 
                               'author': rank2[p]['author.label'],
                               'created': rank2[p]['node.created']}
        else:
            score[uri] = (score[uri]) * w_rank1 + (1/(p+1+k)) * w_rank2


    uri_list = []
    score_list = []
    sorted_title_text = []
    for i in sorted(score, key = score.get, reverse=True):
        uri_list.append(i)
        score_list.append(score[i])
        sorted_title_text.append(title_text[i])
    return (sorted_title_text, score_list)



In [112]:
def busca_grafo_hibrida(query_text):
    results_vector = busca_grafo_vetor(query_text)
    results_text = busca_grafo_fulltext(query_text)
    return RFF(results_vector, results_text, w_rank1=1.0, w_rank2=0.5)

In [113]:
def gen_respostas(query_text):

    results = busca_grafo_hibrida(query_text)

    endpoint = os.getenv("GPT35_ENDPOINT")  
    deployment = os.getenv("DEPLOYMENT_NAME", "gpt-35-turbo") 
    subscription_key = os.getenv("OPENAI_API_KEY")   
        
    
            
    client_chat = AzureOpenAI(  
            azure_endpoint=endpoint,  
            api_key=subscription_key,  
            api_version="2024-05-01-preview",  
        )  

    respostas = []

    for n in range(5):
        text = results[0][n]['text']
        titulo = results[0][n]['title']
        autor = results[0][n]['author']
        created = results[0][n]['created']

        completion = client_chat.chat.completions.create(  
        model=deployment,  
        messages=[
            {
            "role": "system",
            "content": "INSTRUCTIONS:\nAnswer the users QUESTION using the CONTEXT text below.\nKeep your answer ground in the facts of the CONTEX.\n Very important: If the CONTEXT doesn’t contain the facts to answer the QUESTION return {NONE}.\n\nCONTEXT:\n" + text
            },
            {
            "role": "user",
            "content": "QUESTION: " + query_text
            }
            ],  
                #past_messages=10,  
                max_tokens=800,  
                temperature=0.7,  
                top_p=0.95,  
                frequency_penalty=0,  
                presence_penalty=0,  
                stop=None,  
                stream=False  
            )  

        #resposta = completion.to_json()
        resposta = completion.to_dict()['choices'][0]['message']['content'] + " Fonte: " + autor + ". " + titulo + ". " + created + "."
        respostas.append(resposta)
        
    return respostas

In [114]:
def summarize_respostas(respostas, personagem):

    fontes = ''
    for r in respostas:
        fontes = fontes + '\n' + r

    endpoint = os.getenv("GPT35_ENDPOINT")  
    deployment = os.getenv("DEPLOYMENT_NAME", "gpt-35-turbo") 
    subscription_key = os.getenv("OPENAI_API_KEY")   
        
    
            
    client_chat = AzureOpenAI(  
            azure_endpoint=endpoint,  
            api_key=subscription_key,  
            api_version="2024-05-01-preview",  
        )  

    completion = client_chat.chat.completions.create(  
    model=deployment,  
    messages=[
        {
        "role": "system",
        "content": "INSTRUCTIONS:\n Sumarize os textos do contexto, referenciando as fontes ao final em uma bibliografia. Não mencione nem use como fonte os textos que não falem sobre " + personagem + "."

        },
        {
        "role": "user",
        "content": "CONTEXTO: " + fontes
        }
        ],  
            #past_messages=10,  
            max_tokens=800,  
            temperature=0.7,  
            top_p=0.95,  
            frequency_penalty=0,  
            presence_penalty=0,  
            stop=None,  
            stream=False  
        )  

    #resposta = completion.to_json()
    resposta = completion.to_dict()['choices'][0]['message']['content']
        
    return resposta

In [140]:
personagem = 'Achille Mbembe'
query_text = 'Quem foi ' + personagem + ' ?'

respostas = gen_respostas(query_text)
resposta_final = summarize_respostas(respostas, personagem)

In [141]:

respostas

['RESPOSTA: Achille Mbembe é um filósofo camaronês conhecido por seu conceito de necropolítica e suas análises sobre o pós-colonialismo e o racismo. Fonte: Federico, Wellington Luiz. A filosofia política de Achille Mbembe: Entre as influências de Frantz Fanon e Michel Foucault. 2020.',
 'Achille Mbembe é um autor que trata do conflito entre "poder de matar" e "recusa em morrer", e é citado como um dos autores que serão analisados na pesquisa sobre a necropolítica como um projeto político do imaginário, intitulada "O poder de matar e a recusa em morrer: Filopoética Afrodiaspórica como Arquipélago de Libertação". Fonte: Santos, Luís Carlos Ferreira dos. O poder de matar e a recusa em morrer: filopoética afrodiaspórica como arquipélago de libertação.. 2019.',
 '{NONE} - The context does not provide any information about Achille Mbembe. Fonte: Botão, Renato Ubirajara dos Santos UNESP. Para além da nagocracia: a (re)africanização do camdomblé nação angola-congo em São Paulo. 2007.',
 'Achil

In [142]:
print(resposta_final)

Achille Mbembe é um filósofo camaronês conhecido por seu conceito de necropolítica e suas análises sobre o pós-colonialismo e o racismo. Ele é citado em um estudo sobre a necropolítica como projeto político do imaginário e em uma pesquisa sobre a arte afrocentrada de Keila Sankofa. Fontes: Federico, Wellington Luiz. A filosofia política de Achille Mbembe: Entre as influências de Frantz Fanon e Michel Foucault. 2020; Santos, Luís Carlos Ferreira dos. O poder de matar e a recusa em morrer: filopoética afrodiaspórica como arquipélago de libertação.. 2019; Aflitos, Lucas Lopes da Silva. Quem ela pensa que é? corpo, performance, memória e a arte afrocentrada de Keila Sankofa. 2023.


In [ ]:
#texto que será usado na busca
#query_text = 'Nelson Mandela'
personagem = 'Kabengele Munanga'
query_text = 'Quem foi ' + personagem + ' ?'
# Transformando as query em vetor
#query_embedding = get_embeddings(query_text)
query_embedding = get_embeddings_openAI(query_text)[0].embedding

In [ ]:
# Buscando no índice de vetores 
result = graph.query("""
    CALL db.index.vector.queryNodes('Thesis_Embeddings', 5, """ + str(query_embedding) + """)
    YIELD node, score
    RETURN node.title, node.abstract
    """)

contexto = ''
for r in result:
    contexto = contexto + 'Título: ' + r['node.title'] + ' \n'
    contexto = contexto + 'Título: ' + r['node.abstract'] + ' \n \n'
print(contexto)

In [ ]:
%pip install python-dotenv
%pip install openai==1.13.3
%pip install -qU langchain-openai
%pip install neo4j
%pip install langchain langchain-community

In [1]:
from dotenv import load_dotenv
import os
from openai import AzureOpenAI
from langchain_community.graphs import Neo4jGraph
from langchain_openai import AzureChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain.prompts import PromptTemplate
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chains import LLMChain

In [2]:
load_dotenv()

# Azure OpenAI variables
OPENAI_API_VERSION = os.getenv("OPENAI_API_VERSION")
AZURE_OPENAI_ENDPOINT =os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
OPENAI_DEPLOYMENT_NAME = os.getenv("OPENAI_DEPLOYMENT_NAME")

# Neo4j variables
NEO4J_URL = os.getenv("NEO4J_URL")
NEO4J_USERNAME =os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

### Connecting to the graph and making a Cypher query

In [3]:
#Connecting to the graph
graph = Neo4jGraph(
    url=NEO4J_URL,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

In [16]:
#Quering the graph unsing Cypher
result = graph.query("""
MATCH (ner{uri:'tag:stardog:api:kabengele_munanga'})
MATCH p=(thesis)-[r:mentions]->(ner)
RETURN thesis.title, thesis.abstract, thesis.repository
LIMIT 15
""")

In [17]:
result

[{'thesis.title': 'A inserção de alunos imigrantes africanos negros na rede estadual de ensino na cidade de São Paulo (2014-2016)',
  'thesis.abstract': 'This work analyses the insertion of black immigrant students in a secondary school of the state educational system of São Paulo, in São Paulo city, SP, Brazil, among the years of 2014 and 2016. That choice is based on the professional position occupied by the researcher, who works for the same state education system specifically with its racial relations issues. The research aims to describe and evaluate the insertion of those students in the school routine, through identifying the relations among the black African immigrant students, the other students, the teachers, the school administrators and the school staff in general, as well as the relations among cultures in the school environment. To reach those aims, the researcher observed the classes, free times, entrance and leaving of students, how they arrange themselves in the workro

In [6]:
# Printing the graph schema
graph.refresh_schema()
print(graph.schema)

Node properties:
Resource {label: STRING, uri: STRING, family_name: STRING, givenname: STRING, repository: STRING, created: STRING, abstract: STRING, title: STRING, identifier: STRING}
_GraphConfig {_dataTypePropertyLabel: STRING, _subPropertyOfRel: STRING, _classNamePropName: STRING, _handleVocabUris: INTEGER, _applyNeo4jNaming: BOOLEAN, _relNamePropName: STRING, _domainRel: STRING, _keepLangTag: BOOLEAN, _keepCustomDataTypes: BOOLEAN, _handleMultival: INTEGER, _objectPropertyLabel: STRING, _rangeRel: STRING, _classLabel: STRING, _handleRDFTypes: INTEGER, _subClassOfRel: STRING}
Program {uri: STRING, label: STRING}
NamedIndividual {family_name: STRING, label: STRING, uri: STRING, givenname: STRING, acronym: STRING, identifier: STRING, created: STRING, abstract: STRING, title: STRING, repository: STRING}
Department {label: STRING, uri: STRING}
Person {family_name: STRING, label: STRING, givenname: STRING, uri: STRING}
University {label: STRING, acronym: STRING, uri: STRING}
Subject {la

### Using a cypher query to ground the LLM

In [7]:
# Connection to the Azure OpenAI model
client = AzureOpenAI(
  azure_endpoint=AZURE_OPENAI_ENDPOINT, 
  api_key=AZURE_OPENAI_API_KEY,  
  api_version=OPENAI_API_VERSION
)


chat_llm = AzureChatOpenAI(deployment_name=OPENAI_DEPLOYMENT_NAME,temperature=0)

In [22]:
# Using information queried from the knowledge graph
ground_information =  graph.query(""" 
MATCH (ner{uri:'tag:stardog:api:ana_lucia_silva_souza'})
MATCH p=(thesis)-[r:mentions]->(ner)
RETURN thesis.title, thesis.abstract, thesis.repository
LIMIT 10
""")
#ground_information =  graph.query(""" MATCH (n:Wellbore {WellboreName: "31/2-22 S"}) RETURN n;""")


prompt = PromptTemplate(template="""You are a bot assistant having a conversation about academic thesis and dissertations. You should strictly follow the information from your context and always provide the references (title of the document and link for the repository immediately after you mention one fact).

Chat History: {chat_history}
Context: {context}
Question: {question}
""", input_variables=["chat_history", "context", "question"])

In [23]:
memory = ConversationBufferMemory(memory_key="chat_history", input_key="question", return_messages=True)

chat_chain = LLMChain(llm=chat_llm, prompt=prompt, memory=memory) #LLMChain

while True:
    question = input("> ")
    print(question)
    response = chat_chain.invoke({
        "context": ground_information,
        "question": question
        })
    print(response["text"])
    if question == "quit":
        break

Quem foi Ana Lucia Silva Souza?
Ana Lucia Silva Souza é uma autora que desenvolveu a teoria dos Letramentos de Reexistências, utilizada em algumas das teses mencionadas no contexto. Você pode encontrar mais informações sobre ela em: https://repositorio.ufba.br/ri/bitstream/ri/34885/1/Disserta%C3%A7%C3%A3o%20Ana%20L%C3%BAcia%20Silva%20Souza.pdf.
quit
Goodbye! Don't hesitate to ask if you have any more questions.


In [29]:
question = input("> ")
#print (question)


In [7]:
# Connection to the Azure OpenAI model
client = AzureOpenAI(
  azure_endpoint=AZURE_OPENAI_ENDPOINT, 
  api_key=AZURE_OPENAI_API_KEY,  
  api_version=OPENAI_API_VERSION
)


llm = AzureChatOpenAI(deployment_name=OPENAI_DEPLOYMENT_NAME,temperature=0)

### Using LLM to query the graph using GraphCypherQAChain library

In [9]:
# FIRST PROMPT - Transform a natural language query in a Cypher query
CYPHER_GENERATION_TEMPLATE = """
You are an expert Neo4j Developer translating user questions into Cypher to answer questions about the offshore oil and gas industry.
Convert the user's question based on the schema.
Do not include any text except the generated Cypher statement.

Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
The property existence syntax `... exists(variable.property)` is no longer supported. Please use `variable.property IS NOT NULL` instead.

Schema: {schema}
Question: {question}
"""

cypher_generation_prompt = PromptTemplate(
    template=CYPHER_GENERATION_TEMPLATE,
    input_variables=["schema", "question"],
)

# Memory for chat conversation
memory = ConversationBufferMemory(memory_key="chat_history", input_key="query", return_messages=True)

# SECOND PROMPT - Use the Cypher query to answer the user query
cypher_chain = GraphCypherQAChain.from_llm(
    llm,
    graph=graph,
    cypher_prompt=cypher_generation_prompt,
    verbose=True,
    memory=memory
)


# Chat loop
while True:
    question = input("> ")
    response = cypher_chain.invoke({
        "query": question
        })
    
    print(response['result'])



> Entering new GraphCypherQAChain chain...
Generated Cypher:
What are the names of the facilities that are responsible for drilling wellbore with NpdidWellbore '5693-L-008'? 

Cypher statement:
MATCH (:Wellbore {NpdidWellbore: '5693-L-008'})<-[:DRILLED]-(f:Facility) WHERE EXISTS((:Company)-[:IS_RESPONSIBLE_FOR]->(f)) RETURN f.Name


ValueError: Generated Cypher Statement is not valid
{code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'What': expected
  "ALTER"
  "CALL"
  "CREATE"
  "DEALLOCATE"
  "DELETE"
  "DENY"
  "DETACH"
  "DROP"
  "DRYRUN"
  "ENABLE"
  "FINISH"
  "FOREACH"
  "GRANT"
  "INSERT"
  "LOAD"
  "MATCH"
  "MERGE"
  "NODETACH"
  "OPTIONAL"
  "REALLOCATE"
  "REMOVE"
  "RENAME"
  "RETURN"
  "REVOKE"
  "SET"
  "SHOW"
  "START"
  "STOP"
  "TERMINATE"
  "UNWIND"
  "USE"
  "USING"
  "WITH" (line 1, column 1 (offset: 0))
"What are the names of the facilities that are responsible for drilling wellbore with NpdidWellbore '5693-L-008'?"
 ^}

Testando a vetorizacao dos textos

In [23]:
#Quering the graph unsing Cypher
result = graph.query("""
MATCH (t:Thesis)
RETURN t.title, t.abstract
LIMIT 15
""")

result

[{'t.title': '... E Teko e Arandu e produção de subjetividades e educação superior e educações outras...: modos de vida criados e afirmados por Kaiowás e Guaranis',
  't.abstract': 'Esta pesquisa persegue pistas na produção de subjetividades de indígenas que se formaram na Educação Superior e se encontram nas suas comunidades de origem. Tal produção implica abrir-se a vivenciar modos outros de ser e estar no mundo, entendendo que estes devêm no interior de campos de forças que se relacionam, e possibilitam a emergência de existências singulares de vida que subvertem aquelas submetidas a ordens sociais preestabelecidas. Assim, ressaltam-se duas problematizações, uma na direção das afirmações de vida – ou seja das diferentes dimensões da experiencia humana, da busca pelas expressões de sua potência e da valorização da diversidade e individualidade como expressões de vida –, que são produzidas por esses indígenas que, tendo se formado na Educação Superior, continuam a viver nas suas comun

In [25]:
abstract = [i['t.abstract'] for i in result]
abstract

['Esta pesquisa persegue pistas na produção de subjetividades de indígenas que se formaram na Educação Superior e se encontram nas suas comunidades de origem. Tal produção implica abrir-se a vivenciar modos outros de ser e estar no mundo, entendendo que estes devêm no interior de campos de forças que se relacionam, e possibilitam a emergência de existências singulares de vida que subvertem aquelas submetidas a ordens sociais preestabelecidas. Assim, ressaltam-se duas problematizações, uma na direção das afirmações de vida – ou seja das diferentes dimensões da experiencia humana, da busca pelas expressões de sua potência e da valorização da diversidade e individualidade como expressões de vida –, que são produzidas por esses indígenas que, tendo se formado na Educação Superior, continuam a viver nas suas comunidades; e outra, no caminho de indagar por como a Educação Matemática pode operar tanto com esses modos de vida que são produzidos quanto com as teorizações sobre as relações entre

In [4]:
from langchain_openai import OpenAIEmbeddings

In [26]:
#import os
#from openai import AzureOpenAI

#client = AzureOpenAI(
#  api_key = os.getenv("AZURE_OPENAI_API_KEY"),  
#  api_version = "2024-06-01",
#  azure_endpoint =os.getenv("AZURE_OPENAI_ENDPOINT") 
#)

response = client.embeddings.create(
    input = abstract,
    model= "text-embedding-3-small"
)

print(response.model_dump_json(indent=2))

{
  "data": [
    {
      "embedding": [
        0.04626331478357315,
        0.02560325898230076,
        0.02731436863541603,
        0.011481329798698425,
        -0.013963493518531322,
        -0.01917075738310814,
        0.046559061855077744,
        0.05412174016237259,
        -0.02296265959739685,
        -0.017258962616324425,
        0.0074781798757612705,
        -0.00171374948695302,
        -0.04051737114787102,
        -0.021272676065564156,
        0.05551597848534584,
        0.02613138034939766,
        -0.0006707124412059784,
        -0.008196422830224037,
        0.02232891507446766,
        0.031095707789063454,
        0.03567979112267494,
        -0.03113795816898346,
        -0.002278838073834777,
        0.0051095616072416306,
        0.021082552149891853,
        -0.06548688560724258,
        -0.03713740035891533,
        0.032827943563461304,
        -0.006775780115276575,
        0.012706568464636803,
        0.02048049494624138,
        -0.00316079845651984

In [29]:
response.data[2].embedding

[0.06705811619758606,
 0.005367634352296591,
 0.04473194479942322,
 0.015799449756741524,
 0.0018841441487893462,
 -0.012973855249583721,
 0.005357685033231974,
 0.01555071771144867,
 -0.0267635527998209,
 -0.0003179104533046484,
 -0.02274404466152191,
 -0.015341782942414284,
 -0.0548006109893322,
 0.004188645165413618,
 0.0316585972905159,
 0.04278188571333885,
 0.028574319556355476,
 -0.007347539998590946,
 0.02616659551858902,
 0.017729610204696655,
 0.05293014645576477,
 -0.025052277371287346,
 -0.032593827694654465,
 0.002922599669545889,
 -0.025907915085554123,
 -0.021112363785505295,
 -0.030763160437345505,
 0.028514623641967773,
 -0.03635465353727341,
 0.04799530655145645,
 -0.011580957099795341,
 -0.007660942152142525,
 -0.029449855908751488,
 -0.0003584848309401423,
 -0.031957074999809265,
 0.05722823366522789,
 -0.06514785438776016,
 -0.03500155359506607,
 0.03315098583698273,
 -0.04061294347047806,
 -0.009337395429611206,
 0.006825203076004982,
 -0.015948688611388206,
 0.01